## Persona

In [1]:
import importlib
import brikasutils as bu
importlib.reload(bu)
import fb_msg_reader as fb
importlib.reload(fb)
import shared_utils as utils
importlib.reload(utils)
from typing import List
import re


In [2]:
import persona
importlib.reload(persona)

texts_with_elias = [
    "selected-data/elias/message_1.json",
]

texts_with_petyo = [
    "selected-data/petyo/message_1.json",
    "selected-data/petyo/message_2.json",
    "selected-data/petyo/message_3.json",
    "selected-data/petyo/message_4.json",
    "selected-data/petyo/message_5.json",
]

ab = persona.PersonaEncoder()
ab.parse_fb_messages(texts_with_elias, "elias")
ab.parse_fb_messages(texts_with_petyo, "petyo")

ab.filter_chats_empty()
ab.filter_chats_regex(utils.BLACKLIST_CHAT_REGEX_FILTERS)

# Compress names
for nameid, chat in ab.chats.items():
    for msg in chat:  
        msg.sender = "Persona" if msg.sender == "Airidas Brikas" else "Friend"

# Start all chats from 2/3rds
for nameid, chat in ab.chats.items():
    ab.chats[nameid] = chat[int(len(chat)/3 * 2):]

ab.select_chat_limited_by_tokens("elias", 6000)
ab.select_chat_limited_by_tokens("petyo", 6000)
persona_text = ab.output()
bu.quickTXT(persona_text)



Read 1946 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2021-09-13 to 2024-03-06
Messages saved to self.fb_messages['elias']
Read 40036 messages from 5 files. Failed to read 0 messages.
Messages ranged from 2020-08-17 to 2024-03-04
Messages saved to self.fb_messages['petyo']
Filtering
link-filter: 950
react-filter: 30


/Users/twenythree/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Selected chat elias for 5828 (211 messages)
Selected chat petyo for 5997 (402 messages)


## Survey

In [5]:
import survey
importlib.reload(survey)
csv_file = "surveys/survey_kano-model_v1.csv"
my_survey = survey.KanoSurvey(csv_file)

4     What would you say if there were options to de...
5     What would you say if there were NO options to...
6     What would you say if the game had the option ...
7     What would you say if the game did NOT have th...
8     What would you say if the game has good graphics?
9     What would you say if the game had NO good gra...
10    What would you say if the game had an exciting...
11    What would you say if the game did NOT have an...
12    What would you say if there were rewards such ...
13    What would you say if there were NO rewards su...
14    What would you say if the game had realistic g...
15    What would you say if the game does NOT have r...
16    What would you say if the game had a relaxed f...
17    What would you say if the game DON'T have a re...
18    What would you say if the game had a multiplay...
19    What would you say if the game does NOT have m...
20    What would you say if you can loot defeated en...
21    What would you say if you CANNOT loot defe

## Prompt Builder

In [6]:
def userMsg(str) -> dict:
    return {"role": "user", "content": str}
def assistantMsg(str) -> dict:
    return {"role": "assistant", "content": str}
def systemMsg(str) -> dict:
    return {"role": "system", "content": str}

In [11]:
persona_text = "Favorite video games are Rimworld, Minecraft, Age of Empires, 7 Days to Die"


In [18]:
final_prompts = []

for question in my_survey.questions:
    p = [
        userMsg(
            "\n".join([
                "Text below contains extract of data about a persona. Use this data to understanding the likes and prefferences of the persona.",
                persona_text
            ])
        ),
        assistantMsg("Understood. I will answer my questions from the point of view of the persona"),
        userMsg("\n".join([
            f"You are surveyed about video game prefferences. Reply to the statement below with one of the five options: {', '.join(my_survey.POSSIBLE_ANSWERS)}. Your answer should only contain the chosen option. ",
            question
        ])),
    ]
    final_prompts.append(p)

bu.quickJSON(final_prompts, "prompts.json")

## Analysis

In [ ]:
import pandas as pd
from openai import OpenAI
import os

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

results = []
l = len(final_prompts)

for i, (prompt, question) in enumerate(list(zip(final_prompts, my_survey.questions))):
    print(f"{i}/{l}...", end="") # Print progress
    # Send the Request
    full_response = client.chat.completions.create(
        messages=prompt,
        model="gpt-4",
    )
    r = full_response.choices[0].message.content

    # Process response
    if r not in my_survey.POSSIBLE_ANSWERS:
        results.append({'question': question, 'answer': r, 'isValid': False})
        continue
    results.append({'question': question, 'answer': r, 'isValid': True})
    print(f"{question}:\n {r}")



In [20]:
df = pd.DataFrame(results)

air = my_survey.test_answers["airidas"]
eli = my_survey.test_answers["elias"]
df.insert(2, "airidas", air[:len(df)])
df.insert(3, "elias", eli[:len(df)])

df['answer'] = df['answer'].str.lower()
df['airidas'] = df['airidas'].str.lower()
df['elias'] = df['elias'].str.lower()



In [21]:
df

,question,answer,airidas,elias,isValid
0,What would you say if there were options to de...,i like it,i am neutral,i am neutral,True
1,What would you say if there were NO options to...,i can tolerate it,i can tolerate it,i expect it,True
2,What would you say if the game had the option ...,i like it,i expect it,i expect it,True
3,What would you say if the game did NOT have th...,i can tolerate it,i dislike it,i can tolerate it,True
4,What would you say if the game has good graphics?,i like it,i like it,i like it,True
5,What would you say if the game had NO good gra...,i can tolerate it,i am neutral,i can tolerate it,True
6,What would you say if the game had an exciting...,i expect it,i like it,i like it,True
7,What would you say if the game did NOT have an...,i can tolerate it,i can tolerate it,i dislike it,True
8,What would you say if there were rewards such ...,i like it,i like it,i dislike it,True
9,What would you say if there were NO rewards su...,i can tolerate it,i dislike it,i like it,True


In [17]:
df_1a = df.copy()

In [16]:
df.to_csv("1_results.csv")

In [22]:
df_1a["2a"] = df["answer"]

In [23]:
df_1a

,question,answer,airidas,elias,isValid,2a
0,What would you say if there were options to de...,i like it,i am neutral,i am neutral,True,i like it
1,What would you say if there were NO options to...,i can tolerate it,i can tolerate it,i expect it,True,i can tolerate it
2,What would you say if the game had the option ...,i like it,i expect it,i expect it,True,i like it
3,What would you say if the game did NOT have th...,i can tolerate it,i dislike it,i can tolerate it,True,i can tolerate it
4,What would you say if the game has good graphics?,i like it,i like it,i like it,True,i like it
5,What would you say if the game had NO good gra...,i can tolerate it,i am neutral,i can tolerate it,True,i can tolerate it
6,What would you say if the game had an exciting...,i expect it,i like it,i like it,True,i expect it
7,What would you say if the game did NOT have an...,i can tolerate it,i can tolerate it,i dislike it,True,i can tolerate it
8,What would you say if there were rewards such ...,i like it,i like it,i dislike it,True,i like it
9,What would you say if there were NO rewards su...,i can tolerate it,i dislike it,i like it,True,i can tolerate it
